In [4]:
import xmltodict, json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [8]:
data = 'liml/beam1_opti_1.liml'

In [9]:
import xmltodict
import pprint

# Open the file and read the contents
with open(data,  'r', encoding='utf-8') as file:
    my_xml = file.read()

# Use xmltodict to parse and convert 
# the XML document
my_dict = xmltodict.parse(my_xml)

# Print the dictionary
pprint.pprint(my_dict, indent=2)


{ 'liml8': { 'analysis': {'@type': 'S20'},
             'displacement': [ { '@axis': '1',
                                 '@selection': 'Unnamed',
                                 'value': '0'},
                               { '@axis': '2',
                                 '@selection': 'Unnamed(2)',
                                 'value': '0'}],
             'elset': { '@color': '-6710887',
                        '@material': 'Material',
                        '@name': 'Default',
                        'elem': [ { '@eid': '1',
                                    '@nodes': '1243 1269 3 1244',
                                    '@shape': 'quad4'},
                                  { '@eid': '2',
                                    '@nodes': '1 30 31 5',
                                    '@shape': 'quad4'},
                                  { '@eid': '3',
                                    '@nodes': '5 31 32 6',
                                    '@shape': 'quad4'},
         

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [10]:
liml = xmltodict.unparse(my_dict, pretty=True, full_document=False)
liml
fout = open('out.liml', 'w')
fout.write(liml)
fout.close()

In [18]:
elems = my_dict['liml8']['elset']['elem'] #elements
ne = len(elems)

In [19]:
nodes = my_dict['liml8']['node'] #nodes

In [79]:
a = 0.5
rhos = np.zeros(ne)
Es = np.zeros(ne)
E0 = 200e9
n = 2 #3が適当??
nu = 0.3
result_elems = my_dict['liml8']['solution']['results']['elem']
sigmas = np.zeros((ne,3))
# Load stress from each element
lambda_tmp = 0
for i in range(ne):
    rhos[i] = a
    Es[i] = rhos[i]**n * E0
    sigma_x = np.array(list(map(lambda x: x['@stressxx'], result_elems[i]['localnode'])),dtype=np.double).sum()
    sigma_y = np.array(list(map(lambda x: x['@stressyy'], result_elems[i]['localnode'])),dtype=np.double).sum()
    tau_xy = np.array(list(map(lambda x: x['@stressxy'], result_elems[i]['localnode'])),dtype=np.double).sum()
    sigmas[i] = np.array([sigma_x,sigma_y,tau_xy])
    lambda_tmp += nu*np.dot(sigmas[i], sigmas[i].transpose())/rhos[i]**n
    
lambda_bar = -1/ne*(n-1)/E0 * lambda_tmp     

In [89]:
# 与えられたnode idから対応するX,Yの点を返す関数
def findXY(node_id):
    for node in nodes:
        if node['@nid'] == str(node_id):
            return (node['@x'], node['@y'])

In [96]:
# 4点から面積を計算する方法（靴紐の公式））
def shoelace_area(points):
    """
    計算順に並んだ4点の(x, y)座標リスト [(x1, y1), (x2, y2), (x3, y3), (x4, y4)] から面積を求める
    """
    n = len(points)
    area = 0
    for i in range(n):
        j = (i + 1) % n  # 次の点（最後の点は最初の点につなげる）
        area += points[i][0] * points[j][1] - points[i][1] * points[j][0]
    return abs(area) / 2

# ✅ 例: 4点を指定して計算
points = [(1, 1), (4, 1), (4, 5), (1, 5)]  # 長方形
# print(shoelace_area(points))  # 出力: 12.0

In [105]:
# 各要素iの体積viを計算する
vs = np.zeros(ne)
for i in range(ne):
    node_ids = elems[i]['@nodes'].split(' ')
    vertices = np.zeros((4, 2))
    for j in range(4):
        (x,y) = findXY(node_ids[j])
        vertices[j, :] = (x,y)
    vs[i] = shoelace_area(vertices)
V0 = vs.sum()    

In [ ]:
tmp_sum = 0
for i in range(ne):
    tmp_sum += rhos[i] * vs[i]
g = tmp_sum - a*V0
dlambda = 0.1 * lambda_bar
tolerance = 1e-5
if abs(g) < tolerance:
    break
if g<0:
    lmbda = lmbda + dlambda
else:
    lbmda = lmbda - dlambda
    

In [81]:
rhos.sum()

377.1

In [51]:
# Write to LIML file
# FEM on LISA
# Load FEM file
# Set initial lambda

IndentationError: unexpected indent (3532217043.py, line 5)

In [83]:
my_dict['liml8']['solution']

{'analysis': {'@type': 'S20'},
 'node': [{'@nid': '1', '@x': '0', '@y': '0', '@z': '0'},
  {'@nid': '2', '@x': '70', '@y': '0', '@z': '0'},
  {'@nid': '3', '@x': '70', '@y': '40', '@z': '0'},
  {'@nid': '4', '@x': '0', '@y': '40', '@z': '0'},
  {'@nid': '5', '@x': '0', '@y': '1.53846153846154', '@z': '0'},
  {'@nid': '6', '@x': '0', '@y': '3.07692307692308', '@z': '0'},
  {'@nid': '7', '@x': '0', '@y': '4.61538461538462', '@z': '0'},
  {'@nid': '8', '@x': '0', '@y': '6.15384615384615', '@z': '0'},
  {'@nid': '9', '@x': '0', '@y': '7.69230769230769', '@z': '0'},
  {'@nid': '10', '@x': '0', '@y': '9.23076923076923', '@z': '0'},
  {'@nid': '11', '@x': '0', '@y': '10.7692307692308', '@z': '0'},
  {'@nid': '12', '@x': '0', '@y': '12.3076923076923', '@z': '0'},
  {'@nid': '13', '@x': '0', '@y': '13.8461538461538', '@z': '0'},
  {'@nid': '14', '@x': '0', '@y': '15.3846153846154', '@z': '0'},
  {'@nid': '15', '@x': '0', '@y': '16.9230769230769', '@z': '0'},
  {'@nid': '16', '@x': '0', '@y': '1